### 순환 신경망 모델 분류

##### 순서가 있는 데이터, 즉 문장 데이터를 입력해서 문장 흐름에서 패턴을 찾아 분류
##### 텍스트 정보를 입력해서 문장에 대한 특징 정보를 추출 (이미 주어진 단어 특징 벡터를 활용해 모델 학습 x)

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

from tqdm import tqdm

##### 랜덤 시드 고정 : 모델과 샘플링을 하는 모든 랜덤 변수의 상태를 고정하기 위한 것.

In [6]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

In [7]:
DATA_IN_PATH = 'C:/Users/0105l/Desktop/고려대학교/대학교_비교과/2021.1학기/KUS실전연구문제단/자연어처리책 실습/data_in/'
DATA_OUT_PATH = 'C:/Users/0105l/Desktop/고려대학교/대학교_비교과/2021.1학기/KUS실전연구문제단/자연어처리책 실습/data_out/'
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

In [8]:
train_input = np.load(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'rb'))
train_input = pad_sequences(train_input, maxlen=train_input.shape[1])
train_label = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

#### 모델 하이퍼파라미터 정의

In [9]:
model_name = 'rnn_classifier_en'
BATCH_SIZE = 128
NUM_EPOCHS = 5
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs = {'model_name':model_name,
        'vocab_size':prepro_configs['vocab_size'],
        'embedding_dimension': 100,
        'dropout_rate': 0.2,
        'lstm_dimension': 150,
        'dense_dimension': 150,
        'output_dimension':1}

##### 모델 구현

In [11]:
class RNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                         output_dim=kargs['embedding_dimension'])
        self.lstm_1_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'],
                                                return_sequences=True)
        self.lstm_2_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'])
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(units=kargs['dense_dimension'],
                               activation=tf.keras.activations.tanh)
        self.fc2= layers.Dense(units=kargs['output_dimension'],
                              activation=tf.keras.activations.sigmoid)
        
        def call(self, x):
            x = self.embedding(x)
            x = self.dropout(x)
            x = self.lstm_1_layer(x)
            x = self.lstm_2_layer(x)
            x = self.dropout(x)
            x = self.fc1(x)
            x = self.dropout(x)
            x = self.fc2(x)
            
            return x

#### 모델 생성

In [12]:
model = RNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])

#### 모델 학습

In [22]:
# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=1)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)

checkpoint_path = DATA_OUT_PATH + model_name + '/weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

C:/Users/0105l/Desktop/고려대학교/대학교_비교과/2021.1학기/KUS실전연구문제단/자연어처리책 실습/data_out/rnn_classifier_en -- Folder already exists 

